In [1]:
from pygraphblas import *
#from _pygraphblas import lib
import pygraphblas.descriptor
import csv
import sys
import logging
import glob
import operator
import sys
sys.path.append("..")
from loader.data_loader import DataLoader
from algorithms.search import naive_bfs_levels



In [2]:
# Setup logger
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('%(asctime)s %(levelname)-5s %(message)s'))
log = logging.getLogger(__name__)
log.propagate = False
log.addHandler(handler)
log.setLevel(logging.INFO)

### Load data

In [ ]:
data_dir = '../../csvs/o1k/'
data_format = 'csv'
loader = DataLoader(data_dir, data_format)

#vertices, mappings, matrices = loader.load_all_csvs()

person = loader.load_vertex('person')
comment = loader.load_vertex('comment')

replyOf = loader.load_edge('replyOf', comment, comment)
knows = loader.load_edge('knows', person, person)
hasCreator = loader.load_edge('hasCreator', comment, person)



### Queries

In [ ]:
# Query 1
def shortest_distance_over_frequent_communication_paths(person, replyOf, knows, hasCreator, num_of_interactions, person1_id, person2_id):

    person1_id_remapped = person.index2id[person1_id]
    person2_id_remapped = person.index2id[person2_id] 

    hasCreatorTransposed = hasCreator.transpose()

    personA_to_comment2 = hasCreatorTransposed @ replyOf
    
    person_to_person = personA_to_comment2.mxm(hasCreator, mask=knows)
    
    person_to_person_filtered = person_to_person.select(lib.GxB_GT_THUNK, num_of_interactions)
    
    overlay_graph = person_to_person_filtered.pattern()
    if num_of_interactions == -1:
        overlay_graph = knows
        
    levels = naive_bfs_levels(overlay_graph, person1_id_remapped)
    
    
    result = levels[person2_id_remapped] - 1 # Get hop count
    
    return result
    

In [ ]:
x = shortest_distance_over_frequent_communication_paths(person, replyOf, knows, hasCreator, 0 , 58, 402)
print(f'RESULT: {x}')